In [1]:
import openrouteservice
import folium
from keplergl import KeplerGl


# 1. Find the sweet spot, based on 30min bike ride between office and city center

In [2]:

client = openrouteservice.Client(key='5b3ce3597851110001cf6248171a4636f30449aa9e446f0682f74ebb') # Specify your personal API key


In [3]:
loc_dict = {
    'first': { #kantar
        'location': [11.2607037, 44.4841799] #swapped lat,long -> long, lat
    }, 
    'second': { #due torri
        'location': [11.3445425, 44.4942094]
    }
}

# Set up folium map
map1 = folium.Map(tiles='Stamen Toner', location=([44.4942094, 11.3445425]), zoom_start=12)

# Request of isochrones with 25 minute bike ride.
params_iso = {'profile': 'cycling-regular',
              'intervals': [1800], # 1800/60 = 30 mins
              'segments': 1800,
              'attributes': ['total_pop'] # Get population count for isochrones
             }

for name, loc in loc_dict.items():
    params_iso['locations'] = [loc['location']] # Add apartment coords to request parameters
    loc['iso'] = client.isochrones(**params_iso) # Perform isochrone request
    folium.features.GeoJson(loc['iso']).add_to(map1) # Add GeoJson to map
    
    folium.map.Marker(list(reversed(loc['location'])), # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='lightgray',
                                        icon_color='#cc0000',
                                        icon='map-marker',
                                        prefix='fa',
                                       ),
                      popup=name,
                 ).add_to(map1) # Add apartment locations to map

map1


    




we use this sweet spot to trace the area where we look for the available apartments. We go to the website Immobiliare.it, draw the area of interest and scrape the website to take the information needed. 


# 2. Scrape Immobiliare.it using Beautiful Soup

Searching Immobiliare.it, we have the opportunity to draw a polygon directly on the map. We mimick the area found using isochrones and we obtain the url for the search result 

<img src = "search-area.png">

In [4]:
# using this learning content: https://realpython.com/beautiful-soup-web-scraper-python/
# using this learning content: https://learningactors.com/how-to-scrape-multiple-pages-of-a-website-using-a-python-web-scraper/

# HTML scrape
import requests 
from bs4 import BeautifulSoup
from time import sleep
from random import randint

# ETL
import pandas as pd
import numpy as np
import re 

# Postgres DB
from sqlalchemy import create_engine
import psycopg2




In [5]:
# pages = [1, 2, 3]
url = 'https://www.immobiliare.it/ricerca.php?idCategoria=1&idContratto=2&idNazione=IT&criterio=rilevanza&ordine=desc&pag=1&vrt=44.483219,11.351624;44.502811,11.352997;44.528822,11.341324;44.522703,11.305618;44.521234,11.287079;44.491852,11.273003;44.484994,11.263046;44.463191,11.279869;44.447997,11.297035;44.474216,11.312828;44.473726,11.331367;44.469071,11.349907;44.483219,11.351624'
# i = 0
# for page in pages:
#     print(page)
#     while i <= page:
#         print(i)
#         print(re.sub('pag=(\d+)', r'pag='+str(i), url))
#         i += 1
    
# create a bs4 object read in html
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
n_pages = int(soup.find_all( 'li', class_ = 'disabled')[-1].text) # takes the page number at the end of the HTML

    # create variable to store page numbers to iterate over
pages = list(np.arange(1, n_pages, 1)) # create list of pages to iterate over
tmp_results = []
complete_results = []
counter = 0

    # iterate over multiple pages and get the request
j = len(pages) # reassign page otherwise it is recognised as response object and cannot make conditional
while counter <= j:
    print(counter)
    page = requests.get(re.sub('pag=(\d+)', r'pag='+str(counter), url))
    # create a bs4 object read in html
    soup = BeautifulSoup(page.content, 'html.parser')
    sleep(randint(2,10))
    tmp_results.append(soup)
    counter +=1

    # iterate over pages and extract text
for ix, soup in enumerate(tmp_results):
        # here we get the content of each ads. We inspected the HTML structure here: https://webformatter.com/html
    complete_results.append(soup.find_all( 'div', class_ = 'listing-item_body--content'))


    # # there is a problem, too many null values
    # final_results[0]


KeyboardInterrupt: 

In [ ]:

len(complete_results)

In [ ]:
# # this is the website after the search. the url seems to carry the lat/lon coords for the search area. this is useful
# # we could teorethically use this in the future to change the dimensions of the search area
# # there are multiple pages, so I want to scrape all the consecutive pages

# # get the page numbers from the bottom of the first page
# page = requests.get('https://www.immobiliare.it/ricerca.php?idCategoria=1&idContratto=2&idNazione=IT&criterio=rilevanza&ordine=desc&pag=1&vrt=44.483219,11.351624;44.502811,11.352997;44.528822,11.341324;44.522703,11.305618;44.521234,11.287079;44.491852,11.273003;44.484994,11.263046;44.463191,11.279869;44.447997,11.297035;44.474216,11.312828;44.473726,11.331367;44.469071,11.349907;44.483219,11.351624')
# # create a bs4 object read in html
# soup = BeautifulSoup(page.content, 'html.parser')
# n_pages = int(soup.find_all( 'li', class_ = 'disabled')[-1].text)

# # create variable to store page numbers to iterate over
# pages = np.arange(1, n_pages, 1)
# tmp_results = []
# complete_results = []

# # iterate over multiple pages and get the request
# for page in pages: 
#     page = requests.get('https://www.immobiliare.it/ricerca.php?idCategoria=1&idContratto=2&idNazione=IT&criterio=rilevanza&ordine=desc&pag='+str(page)+'&vrt=44.483219,11.351624;44.502811,11.352997;44.528822,11.341324;44.522703,11.305618;44.521234,11.287079;44.491852,11.273003;44.484994,11.263046;44.463191,11.279869;44.447997,11.297035;44.474216,11.312828;44.473726,11.331367;44.469071,11.349907;44.483219,11.351624')
#     # create a bs4 object read in html
#     soup = BeautifulSoup(page.content, 'html.parser')
#     sleep(randint(2,10))
#     tmp_results.append(soup)

# # iterate over pages and extract text
# for ix, soup in enumerate(tmp_results):
#     # here we get the content of each ads. We inspected the HTML structure here: https://webformatter.com/html
#     complete_results.append(soup.find_all( 'div', class_ = 'listing-item_body--content'))


# # # there is a problem, too many null values
# # final_results[0]



In [ ]:
dict_ = {
        'name': [], 
        'summary': [],
        'url': [], 
        'price': [], 
        'rooms': [],
        'sqm': [],
        'baths': [], 
        'floors': []
    }

for ix, page in enumerate(complete_results):
    for elem in page:
        #extract name of listing
        name = elem.find_all('p', class_ ="titolo text-primary")

        # copy name text and if not found default
        for i, val in enumerate(name):
            if val.find('a')['title'] is not None:
                dict_['name'].append(str(val.find('a')['title']))
            else:
                None
#                     dict_['name'].append("not_found")



        # extract short summary
        summary = elem.find_all('p', class_ ="descrizione__truncate")
        for i, val in enumerate(summary):
            if val.text is not None:
                dict_['summary'].append(val.text)
            else:
                None
#                 dict_['summary'].append("not_found")


        #extract the ad link
        if elem.find('a')['href'] is not None:
            dict_['url'].append(elem.find('a')['href'])
        else: 
            None

        #extract the apt.price and trasnfrom to int. when not available we default ot 0
        if elem.find('li', class_ = "lif__item lif__pricing") is not None:
            dict_['price'].append(
                int(re.sub(r"[^a-zA-Z0-9]+", ' ', 
                       elem.find('li', class_ = "lif__item lif__pricing").text.strip()).replace(" ", "")
            )
        )
        else: 
            if elem.find('li', class_ = "lif__item lif__pricing--wrapped").text.strip().replace(" ", "") == "PREZZOSURICHIESTA":
                dict_['price'].append('private_treaty')

            
        # extract floor value 
        if elem.find('abbr', class_ ="text-bold im-abbr") is not None:
            dict_['floors'].append(elem.find('abbr', class_ ="text-bold im-abbr").get('title').strip().replace(" ", ""))
        else: 
            dict_['floors'].append('na')
        
        #extract apt features. the structure is rooms, sqm, baths, floor level
        for i in range(0, len(elem.find_all('span', class_ ="text-bold"))):
            if i == 0:
                if elem.find_all('span', class_ ="text-bold") is not None:
                    dict_['rooms'].append(elem.find_all('span', class_ ="text-bold")[i].text)
                else:
                    dict_['rooms'].append("null"+str(elem))

            if i == 1:
                if elem.find_all('span', class_ ="text-bold") is not None:
                    dict_['sqm'].append(elem.find_all('span', class_ ="text-bold")[i].text)
                else:
                    dict_['sqm'].append("null"+str(elem))
            if i == 2:
                if elem.find_all('span', class_ ="text-bold") is not None:
                    dict_['baths'].append(elem.find_all('span', class_ ="text-bold")[i].text.strip().replace(" ", ""))
                else:
                    dict_['baths'].append("null"+str(elem))
            
#             try: 
#                 if elem.find('abbr', class_ ="text-bold im-abbr").text.strip().replace(" ", "") is not None:  
#                     dict_['floors'].append(elem.find('abbr', class_ ="text-bold im-abbr").text.strip().replace(" ", ""))
#                 else: 
#                     dict_['floors'].append(elem.find('abbr', class_ ="text-bold im-abbr").get('title').strip().replace(" ", ""))
#             except:
#                 dict_['floors'].append(elem.find('abbr', class_ ="text-bold im-abbr").get('title').strip().replace(" ", ""))
                
                    
                
#         if elem.find('abbr', class_ ="text-bold im-abbr") is not None:
#             dict_['floors'].append(elem.find('abbr', class_ ="text-bold im-abbr").text.strip().replace(" ", ""))
#         elif elem.find('abbr', class_ ="text-bold im-abbr").text.strip().replace(" ", "") is None: 
#             dict_['floors'].append(elem.find('abbr', class_ ="text-bold im-abbr").get('title').strip().replace(" ", ""))
#         else: 
#             None
            
# initialize the df and manipulate the data
tmp = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_.items() ]))
tmp['id'] = tmp['name'].replace(" ", "", regex = True).replace(",", "", regex = True)+'_'+tmp['price'].astype(str)+'_'+tmp['sqm']
tmp = tmp[['id', 'name', 'summary', 'price', 'sqm', 'rooms', 'baths', 'floors', 'url']]
    


# # sanity check that the extraction gives good results
# counter_nan = 0
# counter_val = 0
# for k, v in dict_.items():
#     if k == 'floor':
#         for i, val in enumerate(dict_[k]):
#             if val == None:
#                 counter_nan +=1
#             else: 
#                 counter_val +=1
    
# final_results[0][0].find('abbr', class_ ="text-bold im-abbr").get('attr = {'name':'title'}')

In [ ]:
# tmp[tmp['baths'].str.contains("null", na = False)]

tmp[tmp['id'].isnull()].count()




In [ ]:
def extract_immobiliare():
    
    # create tmp lists
    dict_ = {
        'name': [], 
        'summary': [],
        'url': [], 
        'price': [], 
        'rooms': [],
        'sqm': [],
        'baths': [], 
        'floors': []
    }

    for ix, page in enumerate(final_results):
        for elem in page:
            #extract name of listing
            name = elem.find_all('p', class_ ="titolo text-primary")

            # copy name text and if not found default
            for i, val in enumerate(name):
                if val.find('a')['title'] is not None:
                    dict_['name'].append(str(val.find('a')['title']))
                else:
                    None
#                     dict_['name'].append("not_found")



        # extract short summary
        summary = elem.find_all('p', class_ ="descrizione__truncate")
        for i, val in enumerate(summary):
            if val.text is not None:
                dict_['summary'].append(val.text)
            else:
                None
#                 dict_['summary'].append("not_found")


        #extract the ad link
        if elem.find('a')['href'] is not None:
            dict_['url'].append(elem.find('a')['href'])
        else: 
            None

        #extract the apt.price and trasnfrom to int. when not available we default ot 0
        if elem.find('li', class_ = "lif__item lif__pricing") is not None:
            dict_['price'].append(
                int(re.sub(r"[^a-zA-Z0-9]+", ' ', 
                       elem.find('li', class_ = "lif__item lif__pricing").text.strip()).replace(" ", "")
            )
        )
        else: 
            None

        # extract floor value
        if elem.find('abbr', class_ ="text-bold im-abbr") is not None:
            dict_['floors'].append(elem.find('abbr', class_ ="text-bold im-abbr").text.strip().replace(" ", ""))
        else: 
            None

        #extract apt features. the structure is rooms, sqm, baths, floor level
        for i in range(0, len(elem.find_all('span', class_ ="text-bold"))):
            if i == 0:
                if elem.find_all('span', class_ ="text-bold") is not None:
                    dict_['rooms'].append(elem.find_all('span', class_ ="text-bold")[i].text)
                else:
                    dict_['rooms'].append("num_0")

            if i == 1:
                if elem.find_all('span', class_ ="text-bold") is not None:
                    dict_['sqm'].append(elem.find_all('span', class_ ="text-bold")[i].text)
            if i == 2:
                if elem.find_all('span', class_ ="text-bold") is not None:
                    dict_['baths'].append(elem.find_all('span', class_ ="text-bold")[i].text.strip().replace(" ", ""))

    # initialize the df and manipulate the data
    tmp = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_.items() ]))
    tmp['id'] = tmp['name'].replace(" ", "", regex = True).replace(",", "", regex = True)+'_'+tmp['price'].astype(str)+'_'+tmp['sqm']
    tmp = tmp[['id', 'name', 'summary', 'price', 'sqm', 'rooms', 'baths', 'floors', 'url']]
    
    return tmp


def initialize_db_extract_immobiliare():
    """
    This function extract the information based on a fixed HTML structure (valid as of 22Mar2021) and import it to a Postgres DB.
    This function is called only the first time, when the DB is not present. If the DB is present, the function shouldn't be called
    to avoid duplicate insertion.
    """

    # create a connection with the database, we use psycopg2 to create the table
    try:
        conn = psycopg2.connect(database = "postgres", user = "manfredi", password = "manfredi", host = "localhost", port = "5432")
    except:
        print("I am unable to connect to the database") 

    cur = conn.cursor()

    # check wether the table "immobiliare" is already present and initialized. 
    # If not, we run the script to initialize and populate for the first time
    cur.execute("select * from information_schema.tables where table_name=%s", ('immobiliare',))
    check = bool(cur.rowcount)
    
    if check == False: # table doesn't exist yet

        # we initialize the table with the format we need 
        try:
            cur.execute("""
                DROP TABLE IF EXISTS immobiliare;
                CREATE TABLE IF NOT EXISTS immobiliare
                (
                    id varchar(255), 
                    name varchar(255),
                    summary varchar(255),
                    price numeric,
                    sqm numeric, 
                    rooms numeric,
                    baths numeric, 
                    floors numeric,
                    url varchar(255),
                    created_at timestamp without time zone NOT NULL DEFAULT NOW(),
                    updated_at timestamp without time zone DEFAULT NULL,
                    CONSTRAINT immobiliare_key PRIMARY KEY (id)
                )
                WITH (
                    OIDS = FALSE
                )
                TABLESPACE pg_default;
                ALTER TABLE immobiliare
                    OWNER to truer;
                CREATE INDEX immobiliare_id ON immobiliare(id, name, sqm, price);""")
        except:
            print("Something wrong happened!")

        conn.commit() # <--- makes sure the change is shown in the database
        conn.close()
        cur.close()
    
    
        
        tmp = extract_immobiliare()
        # we use sqlalchemy to load the data to Postgres
        engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/postgres' \
            .format('manfredi', # username
                    'manfredi', # password
                    'localhost', # host
                    '5432' # local port
                   ) 
            , echo=False)

        # we load into Postgres table created
        tmp.to_sql('immobiliare', engine, if_exists = 'append', index = False,
                       chunksize = 1000, method = 'multi')
        print('The DB has been initialized with '+str(len(tmp))+' rows')
    else:
        print('The database is already initialized')




In [ ]:
initialize_db_extract_immobiliare()


# 3. Insertion of future entries


In [ ]:
def insert_immobiliare():
    
    # we use sqlalchemy to connect to the DB
    engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/postgres' \
                .format('manfredi', # username
                        'manfredi', # password
                        'localhost', # host
                        '5432' # local port
                       ) 
                , echo=False)
    
    # we pull the ids that are already in the table
    ids = pd.read_sql("""select id from public.immobiliare""", con = engine)

    tmp = extract_immobiliare()
    
    
    # yields the elements in `list_2` that are NOT in `list_1`
    diff_list = np.setdiff1d(list(tmp.id),list(ids['id']))

    # select rows based on diff list items. We will import only rows that are not yet present
    tmp[tmp['id'].isin(diff_list)]
    
    # we load into Postgres table created
    tmp.to_sql('immobiliare', engine, if_exists = 'append', index = False,
                       chunksize = 1000, method = 'multi')
    
    print('Inserted '+str(len(tmp))+' rows')
    

In [ ]:
insert_immobiliare()



In [ ]:
ids = pd.read_sql("""select id from public.immobiliare""", con = engine)

In [ ]:
tmp = extract_immobiliare()
list(tmp['id'])

In [ ]:
diff_list = np.setdiff1d(list(tmp.id),list(ids['id']))